In [101]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [102]:
#combining all data to one dataset
current_directory=Path.cwd()
main_folder_path = f"{current_directory}/data/"
#main_folder_path = "/Users/alexandr/Desktop/HomeTask_08/data/"
dataset = pd.DataFrame()
folders = ["idle","running","stairs","walking"]
list_of_items = list()
for name in folders:
    path = f"{main_folder_path}/{name}"
    if os.path.isdir(path):
        for item in os.listdir(path):
            item_path = os.path.join(path, item)
            if os.path.isfile(item_path) and Path(item_path).suffix == ".csv":
                try:
                    info = pd.read_csv(item_path)
                    info['activity'] = name
                    list_of_items.append(info)
                except Exception as e:
                    print(f"A mistake occured while readinthe file {item_path}: {e}")
                
dataset = pd.concat(list_of_items, ignore_index=True)
dataset.to_csv("general_dataset.csv")

In [103]:
print(dataset["activity"].value_counts(), "\n")

print("Dataset size is:", dataset.shape, "\n")

print("Dataset columns' type:", "\n", dataset.dtypes)

print(dataset.isna().sum())

dataset.head()

activity
running    102240
walking     55500
idle        31170
stairs       4950
Name: count, dtype: int64 

Dataset size is: (193860, 4) 

Dataset columns' type: 
 accelerometer_X    float64
accelerometer_Y    float64
accelerometer_Z    float64
activity            object
dtype: object
accelerometer_X    0
accelerometer_Y    0
accelerometer_Z    0
activity           0
dtype: int64


,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,0.009577,5.937620,7.570466,idle
1,0.086191,6.555324,7.785944,idle
2,0.004788,6.440403,7.139510,idle
3,0.277727,6.430826,7.690176,idle
4,-0.047884,6.239290,7.340623,idle


In [104]:
#standartizing parameters
columns = dataset.columns.tolist()

for column in columns[0:-1]:
    dataset[column]=(dataset[column]-dataset[column].mean())/dataset[column].std()
dataset.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,-0.227722,0.347865,0.801710,idle
1,-0.218607,0.397384,0.831673,idle
2,-0.228292,0.388171,0.741785,idle
3,-0.195818,0.387403,0.818356,idle
4,-0.234559,0.372048,0.769750,idle


In [105]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#encoding classes with numbers 
le = LabelEncoder()
dataset["activity"] = le.fit_transform(dataset["activity"])
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

# dividing data into 2 parts: X(parametrs) and y(target)
X = dataset[["accelerometer_X","accelerometer_Y","accelerometer_Z"]]
y = dataset["activity"]

#splitting all data into train and test arrays
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y)

# To see the mapping of encoded labels to original class names:

print(label_mapping)


{'idle': np.int64(0), 'running': np.int64(1), 'stairs': np.int64(2), 'walking': np.int64(3)}


In [106]:
#from sklearn.preprocessing import StandardScaler
#scaler=StandardScaler()
#X_train=scaler.fit_transform(X_train)
#X_test=scaler.fit_transform(X_test)

In [107]:
dataset.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,-0.227722,0.347865,0.801710,0
1,-0.218607,0.397384,0.831673,0
2,-0.228292,0.388171,0.741785,0
3,-0.195818,0.387403,0.818356,0
4,-0.234559,0.372048,0.769750,0


In [108]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report

In [109]:
#RandomForestClassifier
model_random_tree = RandomForestClassifier(n_estimators=100)
model_random_tree.fit(X_train, y_train)

y_pred = model_random_tree.predict(X_test)


In [110]:
#calculating of the RandomForestClassifier model metrics 
accuracy=accuracy_score(y_test, y_pred)
conf_matrix= confusion_matrix(y_test, y_pred)
precision=precision_score(y_test, y_pred, average='weighted')
recall=recall_score(y_test, y_pred,average='weighted')
f1=f1_score(y_test, y_pred, average='weighted')

print("RandomForestClassifier Accurasy: ",accuracy,"\n")
print("RandomForestClassifier confusion_matrix:","\n",conf_matrix,"\n")
print("RandomForestClassifier Precision: ", precision,"\n")
print("RandomForestClassifier Recall",recall,"\n")
print("RandomForestClassifier f1: ", f1,"\n")

RandomForestClassifier Accurasy:  0.999664706489219 

RandomForestClassifier confusion_matrix: 
 [[ 6234     0     0     0]
 [    1 20447     0     0]
 [    0     4   980     6]
 [    0     2     0 11098]] 

RandomForestClassifier Precision:  0.9996648042026642 

RandomForestClassifier Recall 0.999664706489219 

RandomForestClassifier f1:  0.999664082529714 



In [111]:
#Classification report
print("Classification Report for RandomForestClassifier:")
print(classification_report(y_test, y_pred))

Classification Report for RandomForestClassifier:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6234
           1       1.00      1.00      1.00     20448
           2       1.00      0.99      0.99       990
           3       1.00      1.00      1.00     11100

    accuracy                           1.00     38772
   macro avg       1.00      1.00      1.00     38772
weighted avg       1.00      1.00      1.00     38772



In [112]:
#SV Classifier
model_svc = SVC(kernel="rbf", C=2, max_iter=20000)
model_svc.fit(X_train, y_train)

y_pred_svc = model_svc.predict(X_test)

/Users/alexandr/Library/Python/3.11/lib/python/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [113]:
#calculating of the SVC model metrics 
accuracy = accuracy_score(y_test, y_pred_svc)
conf_matrix=confusion_matrix(y_test, y_pred_svc)
precision = precision_score(y_test, y_pred_svc, average='weighted', zero_division=1)
recall=recall_score(y_test, y_pred_svc, average='weighted', zero_division=1)
f1=f1_score(y_test, y_pred_svc, average='weighted', zero_division=1)

print("SV Classifier Accurasy: ",accuracy,"\n")
print("SV Classifier confusion_matrix:","\n",conf_matrix,"\n")
print("SV Classifier Precision: ", precision,"\n")
print("SV Classifier Recall",recall,"\n")
print("SV Classifier f1: ", f1,"\n")

SV Classifier Accurasy:  0.8900495202723615 

SV Classifier confusion_matrix: 
 [[ 6139    80     0    15]
 [  203 18358     0  1887]
 [    8   224     5   753]
 [   31  1062     0 10007]] 

SV Classifier Precision:  0.8973475169768902 

SV Classifier Recall 0.8900495202723615 

SV Classifier f1:  0.8798993639171709 



In [114]:
#classification report of SVC
print("Classification Report for SV Classifier:")
print(classification_report(y_test, y_pred_svc))

Classification Report for SV Classifier:
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      6234
           1       0.93      0.90      0.91     20448
           2       1.00      0.01      0.01       990
           3       0.79      0.90      0.84     11100

    accuracy                           0.89     38772
   macro avg       0.92      0.70      0.68     38772
weighted avg       0.90      0.89      0.88     38772



In [115]:
#GridSearch classifier
#from sklearn.model_selection import GridSearchCV

#param_grid = {
#    'C': [0.1,1,3,5],
#    'kernel': ['linear','rbf'],
#    'gamma':['scale','auto'],
#    'class_weight': ['balanced'],
#    'max_iter':[10000]
#}

#grid_svm=GridSearchCV(SVC(), param_grid, refit=True, scoring='precision_weighted', cv=5)
#grid_svm.fit(X_train,y_train)

#y_pred_svc=grid_svm.predict(X_test)

In [116]:
#accuracy = accuracy_score(y_test, y_pred_svc)
#conf_matrix=confusion_matrix(y_test, y_pred_svc)
#precision = precision_score(y_test, y_pred_svc, average='weighted', zero_division=1)
#recall=recall_score(y_test, y_pred_svc, average='weighted', zero_division=1)
#f1=f1_score(y_test, y_pred_svc, average='weighted', zero_division=1)

#print("SV Classifier Accurasy: ",accuracy,"\n")
#print("SV Classifier confusion_matrix:","\n",conf_matrix,"\n")
#print("SV Classifier Precision: ", precision,"\n")
#print("SV Classifier Recall",recall,"\n")
#print("SV Classifier f1: ", f1,"\n")

In [117]:
# Inferences. Comparing models before extracting Time domain features:
'''
{'idle': np.int64(0), 'running': np.int64(1), 'stairs': np.int64(2), 'walking': np.int64(3)}

Основные выводы o Classification Report for V Classifier:
Модель хорошо классифицирует классы 0 и 1, демонстрируя высокие значения precision, recall и F1-score.
Класс 2 является проблемным: хотя точность (precision) 100%, но recall очень низок (1%). 
Это указывает на то, что модель почти не находит объекты этого класса, несмотря на высокую точность в случаях, 
когда они предсказаны.
Класс 3 также демонстрирует приемлемые результаты, хотя precision немного ниже по сравнению с другими классами.
Средние показатели (macro avg) значительно снижены из-за низкой производительности по классу 2, 
и взвешенные метрики отражают общее хорошее качество модели для больших классов, но не для маленьких.


Основные выводы Classification Report for RandomForestClassifier:
Модель RandomForestClassifier показывает отличное результаты для всех классов, с точностью, полнотой и f1-score, 
близкими к 1.00.
Класс 2 показывает гораздо более улучшенные результаты по сравнению с моделью SVC — полнота порядка 0.97, что означает, 
что модель находит почти все объекты этого класса.
Общая точность модели составляет 100%, что указывает на отличную работу модели с текущими данными.

Эта модель показывает лучшее качество классификации, чем предыдущая SVC, особенно для класса 2 and class 3. 

'''


"\n{'idle': np.int64(0), 'running': np.int64(1), 'stairs': np.int64(2), 'walking': np.int64(3)}\n\nОсновные выводы o Classification Report for V Classifier:\nМодель хорошо классифицирует классы 0 и 1, демонстрируя высокие значения precision, recall и F1-score.\nКласс 2 является проблемным: хотя точность (precision) 100%, но recall очень низок (1%). \nЭто указывает на то, что модель почти не находит объекты этого класса, несмотря на высокую точность в случаях, \nкогда они предсказаны.\nКласс 3 также демонстрирует приемлемые результаты, хотя precision немного ниже по сравнению с другими классами.\nСредние показатели (macro avg) значительно снижены из-за низкой производительности по классу 2, \nи взвешенные метрики отражают общее хорошее качество модели для больших классов, но не для маленьких.\n\n\nОсновные выводы Classification Report for RandomForestClassifier:\nМодель RandomForestClassifier показывает отличное результаты для всех классов, с точностью, полнотой и f1-score, \nблизкими к

In [118]:
#creating windows and calculating Time domain features using tsfresh 
from tsfresh import extract_features

# Параметры окна
window_size = 4
step_size = 2

# Список для хранения всех временных признаков(time domain features) для каждого окна
all_features = []

# Проходим по каждому классу активности

for activity_class, group in dataset.groupby("activity"):
    for i in range(0, len(group) - window_size +1, step_size):
        window = group.iloc[i:i+window_size].copy()

        # Рассчитываем временные признаки для осей X, Y, Z
        features = {}

        for axis in dataset.columns.tolist()[0:-1]:
            data = window[axis]

            features[f"{axis}_mean"] = data.mean()
            features[f"{axis}_variance"] = data.var()
            features[f"{axis}_std"] = data.std()
            features[f"{axis}_median"] = data.median()
            features[f"{axis}_max"] = data.max()
            features[f"{axis}_min"] = data.min()
            features[f"{axis}_mean"] = data.mean()
            features[f"{axis}_absolute_sum_of_changes"] = np.sum(np.abs(np.diff(data)))
            features[f"{axis}_skewness"] = data.skew()
            features[f"{axis}_kurtosis"] = data.kurtosis()
            features[f"{axis}_quantile_0.25"] = data.quantile(0.25)
            features[f"{axis}_quantile_0.50"] = data.quantile(0.50)
            features[f"{axis}_quantile_0.75"] = data.quantile(0.75)
            features[f"{axis}_root_mean_square"] = np.sqrt(np.mean(data**2))
            #features[f"{axis}_mean_absolute_deviation"] = data.mad()
            #mad
            features[f"{axis}_mean_absolute_deviation"] = np.mean(np.abs(data - np.mean(data)))
            features[f"{axis}_range_count_positive"] = np.sum((data>0) & (data<1e12))
            features[f"{axis}_range_count_negative"] = np.sum((data<0)&(data<-1e12))
            features[f"{axis}_energy"] = np.sum(data**2) / window_size
        features["cross_correlation_XY"] = np.correlate(window["accelerometer_X"], window["accelerometer_Y"])
        features["cross_correlation_XZ"] = np.correlate(window["accelerometer_X"], window["accelerometer_Z"])
        features["cross_correlation_YZ"] = np.correlate(window["accelerometer_Y"], window["accelerometer_Z"])

        # Добавляем идентификатор окна и класс активности
        features["id_class"] = i // step_size + len(all_features)
        features["activity"] = activity_class

        # Добавляем временные признаки окна в общий список
        all_features.append(features)

final_set = pd.DataFrame(all_features)


final_set.head()

,accelerometer_X_mean,accelerometer_X_variance,accelerometer_X_std,accelerometer_X_median,accelerometer_X_max,accelerometer_X_min,accelerometer_X_absolute_sum_of_changes,accelerometer_X_skewness,accelerometer_X_kurtosis,accelerometer_X_quantile_0.25,...,accelerometer_Z_root_mean_square,accelerometer_Z_mean_absolute_deviation,accelerometer_Z_range_count_positive,accelerometer_Z_range_count_negative,accelerometer_Z_energy,cross_correlation_XY,cross_correlation_XZ,cross_correlation_YZ,id_class,activity
0,-0.217610,0.000231,0.015190,-0.223164,-0.195818,-0.228292,0.051275,1.539513,2.087723,-0.227864,...,0.799120,0.028298,4,0,0.638592,[-0.3305637425828743],[-0.693968055799537],[1.2143527453552285],0,0
1,-0.225443,0.000427,0.020662,-0.231425,-0.195818,-0.243104,0.079760,1.489692,2.493862,-0.236695,...,0.734805,0.070245,4,0,0.539938,[-0.3481629029294883],[-0.653407119132167],[1.1250867867602772],2,0
2,-0.227295,0.000359,0.018955,-0.233134,-0.199806,-0.243104,0.051844,1.600646,2.939733,-0.236695,...,0.736771,0.071327,4,0,0.542832,[-0.34335056051164076],[-0.6615942831238294],[1.1004277379434515],4,0
3,-0.208209,0.001146,0.033850,-0.215758,-0.163914,-0.237407,0.141290,0.853102,-1.093821,-0.233134,...,0.799705,0.051602,4,0,0.639527,[-0.30621063581866065],[-0.6689658368842515],[1.1735329062508828],6,0
4,-0.202654,0.000939,0.030648,-0.204648,-0.163914,-0.237407,0.127617,0.360061,0.549085,-0.218179,...,0.798249,0.047774,4,0,0.637201,[-0.2900533480159729],[-0.6511672559221293],[1.1413010861853838],8,0


In [119]:
final_set.isna().sum()

accelerometer_X_mean                       0
accelerometer_X_variance                   0
accelerometer_X_std                        0
accelerometer_X_median                     0
accelerometer_X_max                        0
accelerometer_X_min                        0
accelerometer_X_absolute_sum_of_changes    0
accelerometer_X_skewness                   0
accelerometer_X_kurtosis                   0
accelerometer_X_quantile_0.25              0
accelerometer_X_quantile_0.50              0
accelerometer_X_quantile_0.75              0
accelerometer_X_root_mean_square           0
accelerometer_X_mean_absolute_deviation    0
accelerometer_X_range_count_positive       0
accelerometer_X_range_count_negative       0
accelerometer_X_energy                     0
accelerometer_Y_mean                       0
accelerometer_Y_variance                   0
accelerometer_Y_std                        0
accelerometer_Y_median                     0
accelerometer_Y_max                        0
accelerome

In [120]:
final_set.dtypes

accelerometer_X_mean                       float64
accelerometer_X_variance                   float64
accelerometer_X_std                        float64
accelerometer_X_median                     float64
accelerometer_X_max                        float64
accelerometer_X_min                        float64
accelerometer_X_absolute_sum_of_changes    float64
accelerometer_X_skewness                   float64
accelerometer_X_kurtosis                   float64
accelerometer_X_quantile_0.25              float64
accelerometer_X_quantile_0.50              float64
accelerometer_X_quantile_0.75              float64
accelerometer_X_root_mean_square           float64
accelerometer_X_mean_absolute_deviation    float64
accelerometer_X_range_count_positive         int64
accelerometer_X_range_count_negative         int64
accelerometer_X_energy                     float64
accelerometer_Y_mean                       float64
accelerometer_Y_variance                   float64
accelerometer_Y_std            

In [121]:
final_set[["cross_correlation_YZ","cross_correlation_XZ","cross_correlation_XY"]] = final_set[["cross_correlation_YZ","cross_correlation_XZ","cross_correlation_XY"]].astype(float)
final_set.dtypes

accelerometer_X_mean                       float64
accelerometer_X_variance                   float64
accelerometer_X_std                        float64
accelerometer_X_median                     float64
accelerometer_X_max                        float64
accelerometer_X_min                        float64
accelerometer_X_absolute_sum_of_changes    float64
accelerometer_X_skewness                   float64
accelerometer_X_kurtosis                   float64
accelerometer_X_quantile_0.25              float64
accelerometer_X_quantile_0.50              float64
accelerometer_X_quantile_0.75              float64
accelerometer_X_root_mean_square           float64
accelerometer_X_mean_absolute_deviation    float64
accelerometer_X_range_count_positive         int64
accelerometer_X_range_count_negative         int64
accelerometer_X_energy                     float64
accelerometer_Y_mean                       float64
accelerometer_Y_variance                   float64
accelerometer_Y_std            

In [129]:
final_set.isna().sum()

accelerometer_X_mean                           0
accelerometer_X_variance                       0
accelerometer_X_std                            0
accelerometer_X_median                         0
accelerometer_X_max                            0
accelerometer_X_min                            0
accelerometer_X_absolute_sum_of_changes        0
accelerometer_X_skewness                       0
accelerometer_X_kurtosis                       0
accelerometer_X_quantile_0.25                  0
accelerometer_X_quantile_0.50                  0
accelerometer_X_quantile_0.75                  0
accelerometer_X_root_mean_square               0
accelerometer_X_mean_absolute_deviation        0
accelerometer_X_range_count_positive           0
accelerometer_X_range_count_negative       96926
accelerometer_X_energy                         0
accelerometer_Y_mean                           0
accelerometer_Y_variance                       0
accelerometer_Y_std                            0
accelerometer_Y_medi

In [130]:
final_set = final_set.dropna(axis=1)
final_set.isna().sum()

accelerometer_X_mean                       0
accelerometer_X_variance                   0
accelerometer_X_std                        0
accelerometer_X_median                     0
accelerometer_X_max                        0
accelerometer_X_min                        0
accelerometer_X_absolute_sum_of_changes    0
accelerometer_X_skewness                   0
accelerometer_X_kurtosis                   0
accelerometer_X_quantile_0.25              0
accelerometer_X_quantile_0.50              0
accelerometer_X_quantile_0.75              0
accelerometer_X_root_mean_square           0
accelerometer_X_mean_absolute_deviation    0
accelerometer_X_range_count_positive       0
accelerometer_X_energy                     0
accelerometer_Y_mean                       0
accelerometer_Y_variance                   0
accelerometer_Y_std                        0
accelerometer_Y_median                     0
accelerometer_Y_max                        0
accelerometer_Y_min                        0
accelerome

In [122]:
#Если вам нужно удалить столбцы, в которых все значения равны нулю
#final_set = final_set.loc[:, (final_set !=0).any(axis=0)]
#final_set

In [131]:
#standartizing of parametrs
columns = final_set.columns.tolist()

for column in columns[0:-1]:
    final_set[column]=(final_set[column]-final_set[column].mean())/final_set[column].std()
final_set.head()

,accelerometer_X_mean,accelerometer_X_variance,accelerometer_X_std,accelerometer_X_median,accelerometer_X_max,accelerometer_X_min,accelerometer_X_absolute_sum_of_changes,accelerometer_X_skewness,accelerometer_X_kurtosis,accelerometer_X_quantile_0.25,...,accelerometer_Z_quantile_0.75,accelerometer_Z_root_mean_square,accelerometer_Z_mean_absolute_deviation,accelerometer_Z_range_count_positive,accelerometer_Z_energy,cross_correlation_XY,cross_correlation_XZ,cross_correlation_YZ,id_class,activity
0,-0.339315,-0.734528,-1.231164,-0.299426,-0.824380,0.797985,-1.158246,1.333837,0.717076,0.285162,...,0.819922,-0.234849,-1.221536,1.701331,-0.343520,-0.701523,-0.569216,-0.087095,-1.917989,0
1,-0.351530,-0.734345,-1.221235,-0.313682,-0.824380,0.774151,-1.145207,1.286874,0.872234,0.267939,...,0.754660,-0.383103,-1.117610,1.701331,-0.437291,-0.707067,-0.554994,-0.122124,-1.917932,0
2,-0.354417,-0.734408,-1.224333,-0.316631,-0.827605,0.774151,-1.157985,1.391463,1.042572,0.267939,...,0.749745,-0.378570,-1.114929,1.701331,-0.434541,-0.705551,-0.557865,-0.131800,-1.917875,0
3,-0.324658,-0.733672,-1.197305,-0.286645,-0.798579,0.783318,-1.117044,0.686802,-0.498382,0.274884,...,0.829753,-0.233501,-1.163799,1.701331,-0.342631,-0.693852,-0.560449,-0.103113,-1.917818,0
4,-0.315997,-0.733865,-1.203115,-0.267474,-0.798579,0.783318,-1.123302,0.222044,0.129264,0.304052,...,0.801081,-0.236857,-1.173285,1.701331,-0.344842,-0.688763,-0.554209,-0.115761,-1.917761,0


In [132]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
final_set["activity"] = le.fit_transform(final_set["activity"])

X = final_set.drop(columns=["activity"])
y = final_set["activity"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [133]:
#RandomForestClassifier
model_random_tree = RandomForestClassifier(n_estimators=100)
model_random_tree.fit(X_train, y_train)

y_pred = model_random_tree.predict(X_test)


In [134]:
accuracy=accuracy_score(y_test, y_pred)
conf_matrix= confusion_matrix(y_test, y_pred)
precision=precision_score(y_test, y_pred, average='weighted')
recall=recall_score(y_test, y_pred,average='weighted')
f1=f1_score(y_test, y_pred, average='weighted')

print("RandomForestClassifier Accurasy: ",accuracy,"\n")
print("RandomForestClassifier confusion_matrix:","\n",conf_matrix,"\n")
print("RandomForestClassifier Precision: ", precision,"\n")
print("RandomForestClassifier Recall",recall,"\n")
print("RandomForestClassifier f1: ", f1,"\n")


RandomForestClassifier Accurasy:  0.9992778293613949 

RandomForestClassifier confusion_matrix: 
 [[ 3117     0     0     0]
 [    0 10222     0     2]
 [    1     1   486     7]
 [    0     3     0  5547]] 

RandomForestClassifier Precision:  0.9992783876127332 

RandomForestClassifier Recall 0.9992778293613949 

RandomForestClassifier f1:  0.9992758862415357 



In [127]:
print("Classification Report for V Classifier after extracting time domain features:")
print(classification_report(y_test, y_pred))

Classification Report for V Classifier after extracting time domain features:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3117
           1       1.00      1.00      1.00     10224
           2       0.99      0.98      0.98       495
           3       1.00      1.00      1.00      5550

    accuracy                           1.00     19386
   macro avg       1.00      0.99      1.00     19386
weighted avg       1.00      1.00      1.00     19386



In [135]:
#SV Classifier
model_svc = SVC(kernel="rbf", C=2, max_iter=20000)
model_svc.fit(X_train, y_train)

y_pred_svc = model_svc.predict(X_test)

In [136]:
accuracy = accuracy_score(y_test, y_pred_svc)
conf_matrix=confusion_matrix(y_test, y_pred_svc)
precision = precision_score(y_test, y_pred_svc, average='weighted', zero_division=1)
recall=recall_score(y_test, y_pred_svc, average='weighted', zero_division=1)
f1=f1_score(y_test, y_pred_svc, average='weighted', zero_division=1)

print("SV Classifier Accurasy: ",accuracy,"\n")
print("SV Classifier confusion_matrix:","\n",conf_matrix,"\n")
print("SV Classifier Precision: ", precision,"\n")
print("SV Classifier Recall",recall,"\n")
print("SV Classifier f1: ", f1,"\n")

SV Classifier Accurasy:  0.9952543072320231 

SV Classifier confusion_matrix: 
 [[ 3117     0     0     0]
 [    0 10224     0     0]
 [    1     1   440    53]
 [    0     3    34  5513]] 

SV Classifier Precision:  0.9951845615141321 

SV Classifier Recall 0.9952543072320231 

SV Classifier f1:  0.9952079256457218 



In [137]:
print("Classification Report for SV Classifier after extracting time domain features:")
print(classification_report(y_test, y_pred_svc))

Classification Report for SV Classifier after extracting time domain features:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3117
           1       1.00      1.00      1.00     10224
           2       0.93      0.89      0.91       495
           3       0.99      0.99      0.99      5550

    accuracy                           1.00     19386
   macro avg       0.98      0.97      0.97     19386
weighted avg       1.00      1.00      1.00     19386



In [138]:
# Inferences. Comparing models after extracting Time domain features:
'''
{'idle': np.int64(0), 'running': np.int64(1), 'stairs': np.int64(2), 'walking': np.int64(3)}

Основные выводы Classification Report for SV Classifier:
Очень высокая точность модели для классов 0, 1, и 3, где все метрики практически идеальны.
Для класса два предсказания хуже чем для вышеперечисленных классов, однако они значительно выше чем для этой же модели
Без извлечения временных признаков. По отношению к прошлому результату данной модели без извлечения временных признаков
Когда модель показала recall на уровне 0.01 сейчас модель показывает метрику recall на уровне 0.78, что является существенным
Улучшением работы модели.
Хотя показател recall 0.78 все ещё указывает на то, что модель пропускает некоторые объекты этого класса или путает их с другими классами.
В целом, модель очень эффективна, и ее результаты близки к идеальным, но стоит обратить внимание на класс 2(stairs), 
так как он может быть труднее для классификации. 
Кроме того необходимо отметить, что без извлечения признаков даже при количестве
Итераций 20 000 модель так и не сошлась, о чем была выдана предупреждение. после извлечения признаков данное предупреждение
Отсутствует, что означает что модель нашла оптимальные  веса.


Основные выводы Classification Report for RandomForestClassifier:
Модель достигла практически идеальных результатов после извлечения временных доменных признаков, 
с точностью, полнотой и F1-оценками близкими к 1.0 для всех классов.
Даже классы с меньшей поддержкой (например, класс 2) показали очень высокие метрики.
Извлечение временных доменных признаков дало модели важные дополнительные данные для улучшения классификации.

Сравнение двух моделей:
Можно сказать что классы 0(idle) и 1(running) обе модели предсказывают идеально со стопроцентный точностью. Причём для модели RandomForestClassifier
Извлечение временных признаков не принесло существенного роста в предсказаниях и в метриках, так как они до этого были
На уровне 100%. Для модели SVC извлечения временных признаков дало существенный рост метрик по классам 0 и 1.
Класс 3(walking) модель RandomForestClassifier предсказывает немного лучше чем модель SVC, однако расхождение в предсказаниях
По данному классу не столь критичны.

Наиболее уязвимым местом модели SVC являeтся класс номер 2 (stairs). Анализирую метрике можно сказать что модель RandomForestClassifier
превосходит ee существенно. При предсказании данного класса особенно видна эффективность извлечения временных меток 
и особенно для модели SVC.

Выбирая из двух моделей именно для данных целей(задачи связанные с работой акселерометра) я бы выбрал модель RandomForestClassifier.

'''





"\n{'idle': np.int64(0), 'running': np.int64(1), 'stairs': np.int64(2), 'walking': np.int64(3)}\n\nОсновные выводы Classification Report for SV Classifier:\nОчень высокая точность модели для классов 0, 1, и 3, где все метрики практически идеальны.\nДля класса два предсказания хуже чем для вышеперечисленных классов, однако они значительно выше чем для этой же модели\nБез извлечения временных признаков. По отношению к прошлому результату данной модели без извлечения временных признаков\nКогда модель показала recall на уровне 0.01 сейчас модель показывает метрику recall на уровне 0.78, что является существенным\nУлучшением работы модели.\nХотя показател recall 0.78 все ещё указывает на то, что модель пропускает некоторые объекты этого класса или путает их с другими классами.\nВ целом, модель очень эффективна, и ее результаты близки к идеальным, но стоит обратить внимание на класс 2(stairs), \nтак как он может быть труднее для классификации. \nКроме того необходимо отметить, что без извлеч